# Deploy CodeLlama-7b-Instruct-hf on AWS SageMaker

This notebook demonstrates how to deploy Meta's CodeLlama-7b-Instruct-hf model on Amazon SageMaker for manufacturing code generation use cases.

## Prerequisites
- AWS Account with SageMaker access
- Hugging Face account and token
- IAM role with SageMaker permissions

## 1. Setup and Configuration

In [ ]:
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import json

# Configuration
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

print(f"SageMaker role: {role}")
print(f"Region: {region}")

## 2. Deploy the Model

In [ ]:
# Model configuration
hub = {
    'HF_MODEL_ID': 'codellama/CodeLlama-7b-Instruct-hf',
    'HF_TASK': 'text-generation',
    'MAX_INPUT_LENGTH': '4096',
    'MAX_TOTAL_TOKENS': '8192',
}

# Create Hugging Face Model
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version="4.37",
    pytorch_version="2.1",
    py_version="py310",
    model_server_workers=1,
)

print("Model configuration complete")

In [ ]:
# Deploy the model
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name="codellama-endpoint",
    container_startup_health_check_timeout=600,
)

print(f"Endpoint deployed: {predictor.endpoint_name}")

## 3. Test the Endpoint

In [ ]:
# Test with a manufacturing code generation prompt
prompt = """Write a Python function to control a conveyor belt system with:
- Start/Stop functionality
- Speed control (0-100%)
- Emergency stop
- Position tracking"""

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.95,
        "do_sample": True,
    }
}

response = predictor.predict(payload)
print("\n=== Generated Code ===")
print(response[0]['generated_text'])

## 4. Manufacturing Use Cases

In [ ]:
# Example 1: PLC Ladder Logic
plc_prompt = """Generate ladder logic for a simple start-stop motor control circuit with:
- Start button (normally open)
- Stop button (normally closed)
- Motor contactor
- Overload protection"""

response = predictor.predict({"inputs": plc_prompt, "parameters": {"max_new_tokens": 256}})
print(response[0]['generated_text'])

In [ ]:
# Example 2: Quality Control Script
qc_prompt = """Create a Python script for quality control that:
1. Measures part dimensions using a digital caliper
2. Compares against tolerance specifications
3. Generates pass/fail report
4. Logs results to CSV file"""

response = predictor.predict({"inputs": qc_prompt, "parameters": {"max_new_tokens": 512}})
print(response[0]['generated_text'])

## 5. Cleanup (Optional)

In [ ]:
# Delete the endpoint to avoid charges
# predictor.delete_endpoint()
# print("Endpoint deleted")